# Training Unified Roof Analysis Model on SageMaker

This notebook demonstrates training our unified roof analysis model using two datasets:

1. RID (Roof Information Dataset)
   - Provides detailed roof segmentation
   - Has ridge, valley, and eave lines
   - Includes depth information

2. Roofline-Extraction Dataset
   - Focuses on 3D building reconstruction
   - Has ridge, hip, and valley lines
   - Includes depth maps

## Steps:
1. Set up SageMaker environment
2. Prepare and verify datasets
3. Configure and launch training
4. Monitor progress
5. Download and validate results

In [ ]:
import os
import json
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
import boto3
from pathlib import Path

# Initialize SageMaker session
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# S3 bucket for training data and model artifacts
bucket = 'sagemaker-us-east-2-575108929659'
prefix = 'unified-roof-model'

print(f"Using bucket: {bucket}")

## 2. Prepare and Verify Datasets

Our datasets are already in S3 at these locations:
- RID: s3://sagemaker-us-east-2-575108929659/roof-data/RID/
- Roofline: s3://sagemaker-us-east-2-575108929659/roof-data/Roofline/

In [ ]:
# Define dataset paths
dataset_paths = {
    'rid': f's3://{bucket}/roof-data/RID',
    'roofline': f's3://{bucket}/roof-data/Roofline'
}

# Verify datasets exist
s3 = boto3.client('s3')

def check_s3_path(s3_path):
    path_parts = s3_path.replace('s3://', '').split('/')
    bucket = path_parts[0]
    prefix = '/'.join(path_parts[1:])
    
    response = s3.list_objects_v2(
        Bucket=bucket,
        Prefix=prefix,
        MaxKeys=1
    )
    return 'Contents' in response

for dataset, path in dataset_paths.items():
    exists = check_s3_path(path)
    print(f"{dataset}: {'✓' if exists else '✗'} ({path})")

## 3. Configure Training Job

In [ ]:
# Configure hyperparameters
hyperparameters = {
    'epochs-per-dataset': 20,  # Train longer on each dataset
    'batch-size': 16,
    'learning-rate': 0.001,
    'num-workers': 4,
    'num-classes': 12,  # From our unified class mapping
    'datasets': 'rid,roofline'  # Only use RID and Roofline
}

# Create PyTorch estimator
estimator = PyTorch(
    entry_point='sagemaker_train.py',
    source_dir='src',
    role=role,
    framework_version='2.0.1',
    py_version='py39',
    instance_count=1,
    instance_type='ml.p3.2xlarge',  # GPU instance
    hyperparameters=hyperparameters,
    output_path=f's3://{bucket}/{prefix}/output',
    code_location=f's3://{bucket}/{prefix}/code',
    metric_definitions=[
        {'Name': 'train:loss', 'Regex': 'Training Loss: ([0-9\.]+)'},
        {'Name': 'val:loss', 'Regex': 'Validation Loss: ([0-9\.]+)'},
        {'Name': 'segments:loss', 'Regex': 'segments: ([0-9\.]+)'},
        {'Name': 'lines:loss', 'Regex': 'lines: ([0-9\.]+)'},
        {'Name': 'depth:loss', 'Regex': 'depth: ([0-9\.]+)'}
    ],
    enable_sagemaker_metrics=True
)

## 4. Launch Training Job

In [ ]:
# Start training
estimator.fit({
    'rid': dataset_paths['rid'],
    'roofline': dataset_paths['roofline']
})

## 5. Monitor Training Progress

In [ ]:
# Get training job name
training_job_name = estimator.latest_training_job.name
print(f'Training job name: {training_job_name}')

# Get CloudWatch metrics URL
region = sagemaker_session.boto_region_name
cloudwatch_url = f'https://{region}.console.aws.amazon.com/cloudwatch/home?region={region}#metricsV2:graph=~(metrics~(~(~\'AWS*2fSageMaker~\'TrainingJobMetrics~\'TrainingJobName~\'{training_job_name}~\'metric~\'train*3aloss))~view~\'timeSeries~stacked~false~region~\'{region}~stat~\'Average~period~60)'
print(f'\nView metrics at: {cloudwatch_url}')

## 6. Download and Validate Results

In [ ]:
# Download model artifacts
model_artifacts = estimator.model_data
print(f'Model artifacts saved to: {model_artifacts}')

# Download results
results_path = f's3://{bucket}/{prefix}/output/results'
!aws s3 cp {results_path} ./results --recursive